In [1]:
# %load_ext lab_black

# Análise de Artigos em HTML da coleção SciELO Brasil e seus DOIs

# Tópicos

[1 - Introdução](#1-\--Introdução)

  [1.1 - Definição do problema](#1.1---Definição-do-Problema)

[2 - Extração de dados do ArticleMeta](#2---Extração-de-dados-do-ArticleMeta)

  [2.1 - Detecção de artigos sem DOI](#2.1---Detecção-de-artigos-sem-DOI)
  
[3 - Resulução de DOI](#3---Resolução-de-DOI)

   [3.1 - Artigos sem DOI no Articlemeta mas com DOI registrado no Crossref](#3.1---Artigos-sem-DOI-no-Articlemeta-mas-com-DOI-registrado-no-Crossref)

   [3.2 - Artigos sem DOI no Articlemeta e no Crossref](#3.2---Artigos-sem-DOI-no-Articlemeta-e-no-Crossref)

[4. Erros ao resolver URLs](#4---Erros-ao-resolver-URLs)

  [4.1 - Artigos publicados no SciELO respondendo com status 500](#4.1-Artigos-publicados-no-SciELO-respondendo-com-status-500)
  
  [ 4.2 - Documentos publicados no SciELO respondendo com status 404](#4.2---Documentos-publicados-no-SciELO-respondendo-com-status-404)

# 1 - Introdução

Este notebook realiza uma análise exploratória utilizando os dados de artigos em `HTML` registrados no ArticleMeta. Durante o desenvolvimento desta atividade foram gerados scripts `python` para capturar e gravar informações referentes aos artigos.

A partir do conjunto de informação entre **artigo e DOI** foram executadas chamadas de resolução de URLs ao serviço http://doi.org. As respostas para estas chamadas produziu um massa de dados com `263.723` linhas, cada linha representa o estado de resolução de DOI de cada artigo.

Dentre os mais de duzentos mil artigos trabalhados, foi possível observar que apróximadamente `1500` não possuem DOI registrados no `Crossref` ou que pelo menos não foi possível detectar/conhecer estes DOIs. Também foi observado que alguns artigos estão com problemas de resolução de URLs na própria coleção SciELO Brasil.


**PS: É possível realizar o download dos datasets pelo [link](https://drive.google.com/open?id=1iyVX4Q45Ig8SQkpaQKXSWqun5GmiHO1r)**.

## 1.1 - Definição do Problema

Esta tarefa foi iniciada a partir da [issue](https://github.com/scieloorg/articles_meta/issues/173), onde o usuário requer que seja possível o registro de DOI para artigos em `HTML`. O requerente estipula que alguns artigos não possuem DOI registrado e também afirma que atualmente não é possível saber **SE** um DOI está ativo ou não.

Partindo do que foi exposto na pelo requerente, alguns pontos foram elencados podemos nos deparar quando acessamos o DOI registrado no ArticleMeta: 

1. `O artigo possui DOI mas no ArticleMeta não está populado`.
2. `O artigo possui DOI no ArticleMeta mas não está registrado no crossref`.
3. O Artigo não possui DOI e no ArticleMeta não está populado (é preciso garantir que ele não possui DOI).
4. O artigo não possui DOI no ArticleMeta e nem no site (http://www.scielo.br) mas possui na API do Crossref.
5. O Artigo possui DOI no ArticleMeta e está acessível no http://doi.org e no http://www.scielo.br

Todos os pontos acima são importantes, contudo os pontos **1**, **2** serão considerados mais importantes nesta fase de investigação visto que se alinham diretamente com o que foi requisitado.

# 2 - Extração de dados do ArticleMeta

Foi construído um script `python` para extrair todos os artigos em `HTML` e seus respectivos `DOIs`. Para executar o script `scripts/extract_doi_from_articlemeta.py` é necessário entrar com uma lista de `PIDs` (HTML) para realizar a extração.

In [2]:
# Importando as bibliotecas necessárias para executar este script
import pandas as pd
import numpy as np

Ao término do script `scripts/extract_doi_from_articlemeta.py` devemos ter em mãos um arquivo `CSV` com as seguintes colunas:
1. PID
2. DOI
3. Found
4. Collection
5. Error

As colunas `PID` e `DOI` são auto explicativas, foram adicionadas no dataset para possibilitar, se necessário, o cruzamento de dados com outros datasets. Já a coluna `Found` determina que o `DOI` foi encontrado na base do `ArticleMeta`, a coluna `Collection` informa sobre a coleção de publicação do artigo e a coluna `Error` guarda eventuais strings de erros que podem ter ocorridos durante a extração dos `DOIs`. 

In [3]:
df_articlemeta = pd.read_csv(
    "resultados/articlemeta.csv",
    sep=";",
    names=["pid", "doi", "found", "collection", "error"],
    na_filter=False,
)

In [4]:
df_articlemeta.shape

(263723, 5)

## 2.1 - Detecção de artigos sem DOI

A princípio gostaríamos de saber quantos artigos no `ArticleMeta` não possuem `DOI`. Vejamos:

In [5]:
df_articlemeta[df_articlemeta["found"] == 0].reset_index(drop=True).iloc[:, 0:2]

,pid,doi
0,S0102-67202011000200001,
1,S2176-94512010010200002,
2,S2176-94512010010300001,
3,S0034-71671992000200029,
4,S1413-81232009010900001,
...,...,...
3671,S0100-55022013000200015,
3672,S0100-55022013000200017,
3673,S0100-55022013000200016,
3674,S0100-55022013000200018,


Temos então `3.676` artigos sem `DOI` registrado no `ArticleMeta`. Este é o problema 1 (`O artigo possui DOI mas no ArticleMeta não está populado`).

In [6]:
data = df_articlemeta.groupby("found", as_index=True)["pid"].count()
print("Proporção de artigos sem DOI %0.2f%%." % (data[0] / data[1] * 100))

Proporção de artigos sem DOI 1.41%.


# 3 - Resolução de DOI

Nesta fase da investigação nós executamos o script `scripts/access_doi_from_doiorg.py` passando como entrada o `CSV` produzido pelo script `scripts/extract_doi_from_articlemeta.py`. A execução do script é potencialmente demorada (~ 18 horas) uma vez que não devemos abrir conexões indiscriminadamente para outros serviços.

Ao término deste tópico teremos o resultado das resoluções de URL de `DOI` para todos os `263.723` artigos. O arquivo gerado possuirá as seguintes colunas:

1. PID
2. DOI
3. Found by doi
4. Found by generic doi
5. URL
6. Error

A coluna `found_by_doi` indica se a resolução de URL ocorreu com sucesso a partir do `DOI` extraído do `ArticleMeta`. Já a coluna `found_by_generic_doi` indica que a primeira tentativa de resolução falhou, seja porque o `DOI` registrado no `ArticleMeta` está errado ou não foi registrado no `Crossref`, então foi tentada uma nova resolução com um `DOI genérico` (Prefixo ScIELO + PID).

A coluna `URL` indica qual foi a URL de destino resultante da resolução via http://doi.org/[DOI]. E a coluna erro indica os erros que podem acontecer durante todo esse processo.

In [7]:
df_doi = pd.read_csv(
    "resultados/doi.csv",
    delimiter=";",
    names=["pid", "doi", "found_by_doi", "found_by_generic_doi", "url", "error"],
    dtype={
        "pid": str,
        "doi": str,
        "found_by_doi": int,
        "found_by_generic_doi": int,
        "url": str,
        "error": str,
    },
    na_filter=False,
)

## 3.1 - Artigos sem DOI no Articlemeta mas com DOI registrado no Crossref

Agora que possuímos todas as resoluções de URL nós podemos verificar quais documentos foram encontrados via `DOI genérico`. Vejamos:

In [8]:
df_doi[(df_doi["found_by_generic_doi"] == 1)].reset_index(drop=True)[
    ["pid", "doi", "url"]
]

,pid,doi,url
0,S1413-24782013000400014,10.1590/S1413-24782013000400014,http://www.scielo.br/scielo.php?script=sci_art...
1,S0066-782X2010000700001,10.1590/S0066-782X2010000700001,http://www.scielo.br/scielo.php?script=sci_art...
2,S0102-69922012000200013,10.1590/S0102-69922012000200013,http://www.scielo.br/scielo.php?script=sci_art...
3,S0102-69922012000200012,10.1590/S0102-69922012000200012,http://www.scielo.br/scielo.php?script=sci_art...
4,S0102-69922012000200014,10.1590/S0102-69922012000200014,http://www.scielo.br/scielo.php?script=sci_art...
...,...,...,...
3324,S1984-70332013000200007,10.1590/S1984-70332013000200007,http://www.scielo.br/scielo.php?script=sci_art...
3325,S1984-70332013000200008,10.1590/S1984-70332013000200008,http://www.scielo.br/scielo.php?script=sci_art...
3326,S1984-70332013000200009,10.1590/S1984-70332013000200009,http://www.scielo.br/scielo.php?script=sci_art...
3327,S1984-70332013000200010,10.1590/S1984-70332013000200010,http://www.scielo.br/scielo.php?script=sci_art...


Temos no total `3.329` artigos "recuperados" via `DOI genérico`. Este tópico então demonstra quais artigos foram recuperados do problema 1 (`O artigo possui DOI mas no ArticleMeta não está populado`). 

<!-- Sabendo-se então que o total de artigos com `DOI` desconhecido pelo `ArticleMeta` é de `3.676` então é possível concluir que destes, apenas `347` artigos não possuem registro de `DOI` no `Crossref` (detectado a partir da metodologia deste experimento).
 -->

## 3.2 - Artigos com DOI no Articlemeta e sem registro no Crossref

Neste tópico é possível observar quais documento possuem DOI no `ArticleMeta` mas não estão registrados no `Crossref`. 

É interessante notar que alguns destes casos tem relação direta com a escrita errada do **DOI**, por exemplo para o PID `S1980-00372012000300003` o DOI correto é `10.5007//1980-0037.2012v14n3p264` sem as duas barras. É necessário realizar uma pesquisa mais detalhada de forma manual para estes casos.

In [9]:
df_doi[
    (df_doi["found_by_doi"] == 0)
    & (df_doi["found_by_generic_doi"] == 0)
    & (df_doi["doi"] != "")
].reset_index(drop=True)

,pid,doi,found_by_doi,found_by_generic_doi,url,error
0,S0066-782X2013000500003,10.1590/S0066-782X2013005000025,0,0,,
1,S0103-97332010000400004,10.1590/S0103-97332010000400004,0,0,,
2,S0066-782X2011001100013,10.1590/S0066-782X2011001100013,0,0,,
3,S0066-782X2011001100008,10.1590/S0066-782X2011001100008,0,0,,
4,S0066-782X2011001100014,10.1590/S0066-782X2011001100014,0,0,,
...,...,...,...,...,...,...
1522,S0103-97332010000300002,10.1590/S0103-97332010000300002,0,0,,
1523,S0103-97332010000300001,10.1590/S0103-97332010000300001,0,0,,
1524,S0103-97332010000300003,10.1590/S0103-97332010000300003,0,0,,
1525,S1980-00372012000300002,10.5007//1980-0037.2012v14n3p254,0,0,,


De um total de `1.527` artigos sem registro no `Crossref` nós temos `1.338` documentos com prefixo SciELO. É possível que grande parte desses `DOIs` não possam ser reenviados porque o SciELO não é mais o `publisher` autorizado, neste caso é preciso verificar quais artigos estão nesta situação.

In [10]:
df_doi[
    (df_doi["found_by_doi"] == 0)
    & (df_doi["found_by_generic_doi"] == 0)
    & (df_doi["doi"].str.contains("10.1590"))
].reset_index(drop=True)

,pid,doi,found_by_doi,found_by_generic_doi,url,error
0,S0066-782X2013000500003,10.1590/S0066-782X2013005000025,0,0,,
1,S0103-97332010000400004,10.1590/S0103-97332010000400004,0,0,,
2,S0066-782X2011001100013,10.1590/S0066-782X2011001100013,0,0,,
3,S0066-782X2011001100008,10.1590/S0066-782X2011001100008,0,0,,
4,S0066-782X2011001100014,10.1590/S0066-782X2011001100014,0,0,,
...,...,...,...,...,...,...
1333,S0103-97332010000200004,10.1590/S0103-97332010000200004,0,0,,
1334,S0103-97332010000200017,10.1590/S0103-97332010000200017,0,0,,
1335,S0103-97332010000300002,10.1590/S0103-97332010000300002,0,0,,
1336,S0103-97332010000300001,10.1590/S0103-97332010000300001,0,0,,


# 4 - Erros ao resolver URLs

Foi possível observar alguns errors durante a execução dos passos anteriores. A principio `DOIs` estão redirecionando para o endereço `test.scielo.br`, é necessário alguma intervenção manual a fim de modificar a URL para a qual o `DOI` direciona.

In [11]:
df_doi[df_doi["error"].str.contains("test.scielo")].reset_index(drop=True)[
    ["pid", "doi", "error"]
]

,pid,doi,error
0,S0100-84041999000500011,10.1590/S0100-84041999000500011,Cannot connect to host test.scielo.br:80 ssl:d...
1,S0100-84042004000200003,10.1590/S0100-84042004000200003,Cannot connect to host test.scielo.br:80 ssl:d...
2,S0100-84042004000100002,10.1590/S0100-84042004000100002,Cannot connect to host test.scielo.br:80 ssl:d...
3,S0100-40421999000100001,10.1590/S0100-40421999000100001,Cannot connect to host test.scielo.br:80 ssl:d...
4,S0100-40421999000600001,10.1590/S0100-40421999000600001,Cannot connect to host test.scielo.br:80 ssl:d...
5,S0100-40422000000300001,10.1590/S0100-40422000000300001,Cannot connect to host test.scielo.br:80 ssl:d...
6,S1516-635X2000000100004,10.1590/S1516-635X2000000100004,Cannot connect to host test.scielo.br:80 ssl:d...
7,S1516-635X2004000200007,10.1590/S1516-635X2004000200007,Cannot connect to host test.scielo.br:80 ssl:d...
8,S1519-566X2001000200001,10.1590/S1519-566X2001000200001,Cannot connect to host test.scielo.br:80 ssl:d...
9,S1413-73722003000300006,10.1590/S1413-73722003000300006,Cannot connect to host test.scielo.br:80 ssl:d...


## 4.1 Artigos publicados no SciELO respondendo com status 500

Foi possível observar que alguns artigos da coleção brasil não estão abrindo, o site acaba respondendo com o código `500`, indicando assim algum problema no site.

In [12]:
scielo_status_500 = df_doi[
    (df_doi["error"].str.contains("Status code 500"))
    & (df_doi["url"].str.contains("scielo"))
].reset_index(drop=True)[["pid", "doi", "url"]]
scielo_status_500

,pid,doi,url
0,S0100-46702005000200009,10.1590/S0100-46702005000200009,http://www.scielo.br/scielo.php?script=sci_art...
1,S0100-40422011001000010,10.1590/S0100-40422011001000010,http://www.scielo.br/scielo.php?script=sci_art...
2,S0103-97332001000300027,10.1590/S0103-97332001000300027,http://www.scielo.br/scielo.php?script=sci_art...
3,S1678-69712012000600006,10.1590/S1678-69712012000600006,http://www.scielo.br/scielo.php?script=sci_art...
4,S1413-81232003000300007,10.1590/S1413-81232003000300007,http://www.scielo.br/scielo.php?script=sci_art...
...,...,...,...
323,S0100-736X1999000200001,10.1590/S0100-736X1999000200001,http://www.scielo.br/scielo.php?script=sci_art...
324,S0100-879X1999000600011,10.1590/S0100-879X1999000600011,http://www.scielo.br/scielo.php?script=sci_art...
325,S0102-695X2005000100006,10.1590/S0102-695X2005000100006,http://www.scielo.br/scielo.php?script=sci_art...
326,S1678-69712009000100006,10.1590/S1678-69712009000100006,http://www.scielo.br/scielo.php?script=sci_art...


In [13]:
scielo_status_500.shape

(328, 3)

## 4.2 - Documentos publicados no SciELO respondendo com status 404

In [14]:
df_doi[
    (df_doi["error"].str.contains("Status code 404"))
    & (df_doi["url"].str.contains("scielo"))
].reset_index(drop=True)[["pid", "doi", "url"]]

,pid,doi,url
0,S1413-81232012010200001,10.1590/S1413-81232012010200001,https://scielosp.org/article/csc/2012.v17n1/o1...
1,S1413-81232011010500001,10.1590/S1413-81232011010500001,https://scielosp.org/article/csc/2011.v16n11/o...
2,S1413-81232012010500001,10.1590/S1413-81232012010500001,https://scielosp.org/article/csc/2012.v17n4/o1...
3,S1413-81232012010600001,10.1590/S1413-81232012010600001,https://scielosp.org/article/csc/2012.v17n5/o1...
4,S1413-81232012010700001,10.1590/S1413-81232012010700001,https://scielosp.org/article/csc/2012.v17n6/o1...
5,S1413-81232012010400001,10.1590/S1413-81232012010400001,https://scielosp.org/article/csc/2012.v17n3/o1...
6,S1413-81232012011200001,10.1590/S1413-81232012011200001,https://scielosp.org/article/csc/2012.v17n11/o...
7,S1413-81232012011300001,10.1590/S1413-81232012011300001,https://scielosp.org/article/csc/2012.v17n12/o...
8,S1413-81232012011000001,10.1590/S1413-81232012011000001,https://scielosp.org/article/csc/2012.v17n9/o1...
